In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import ctx
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import class from Python CRUD module
from DogFindr import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Username/Password for MongoDB AAC database user
username = 'aacuser'
password = 'Mypassword'

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Send read method empty query to return every document from database
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# Debug for testing database loading correctly
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################

# Create Dash object
app = JupyterDash('Project Two - Michael Devin Moore')

# Load Grazioso Logo and encode the image for use in the HTML output
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Establish web page layout
# Intro and Author banners displayed at top center followed by header break
# Buttons to click to apply Grazioso rescue animal filters
# Create table that has selectable rows, manually filterable columns, 10 rows per page
# Break and Header break
app.layout = html.Div([
    
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Application banner
    html.Center(html.B(html.H1('Welcome To DogFindr!'))),
    
    # Author banner
    html.Center(html.H2('By Michael Devin Moore')),
    
    # Line Break
    html.Hr(),
    
    # Buttons to click to apply Grazioso rescue animal filters
    html.Div(
        className = 'buttonRow',
        style = {'display' : 'flex'},
        children = [
            html.Button('Water Rescue', id = 'buttonOne', n_clicks = 0),
            html.Button('Mountain Rescue', id = 'buttonTwo', n_clicks = 0),
            html.Button('Disaster Rescue', id = 'buttonThree', n_clicks = 0),
            html.Button('Reset', id = 'buttonFour', n_clicks = 0)
        ]
    ),
    
    # Tabke that has selectable rows, manually filterable columns, sortable columns, and 10 rows per page
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data = df.to_dict('records'),
        editable = False,
        filter_action = 'native',
        sort_action = 'native',
        sort_mode = 'multi',
        row_selectable = 'single',
        row_deletable = False,
        selected_columns = [],
        selected_rows = [0],
        page_action = 'native',
        page_current = 0,
        page_size = 10
    ),
    
    # Break
    html.Br(),
    
    # Line Break
    html.Hr(),

# This sets up the dashboard so that histogram, map, and Grazioso logo are presented in a row respectively
    html.Div(
        className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            ),
            html.A([
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style = {'height': '100%', 'width': '100%', 'float': 'right'}),
            ], href = 'https://www.snhu.edu/'
            )
        ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################

# This callback creates the functionality of the filter buttons at the top of the page
# Each button will cause different filtering, or resetting of the filters for 'Reset'
# Each button causes a new dataset to be gathered based off of the respective rescue animal group
# and returned and displayed on the data table
@app.callback(
    Output('datatable-id','data'),
    [Input('buttonOne', 'n_clicks'),
     Input('buttonTwo', 'n_clicks'),
     Input('buttonThree', 'n_clicks'),
     Input('buttonFour', 'n_clicks'),
    ])
def update_dashboard(button1, button2, button3, button4):
    df = pd.DataFrame.from_records(db.read({}))
    
    buttonClicked = ctx.triggered_id
    
    if(buttonClicked == 'buttonOne'):
        waterRescueList = ['Chesa Bay Retr', 'Labrador Retriever Mix', 'Newfoundland']
        df = pd.DataFrame.from_records(db.read(
            {'$and': [{'animal_type' : 'Dog'}, {'breed': {'$in': waterRescueList}}, {'sex_upon_outcome': 'Intact Female'},
                {'age_upon_outcome_in_weeks': {'$gt': 26}}, {'age_upon_outcome_in_weeks': {'$lt': 156}}]}))
    elif(buttonClicked == 'buttonTwo'):
        mountainRescueList = ['Alaskan Malamute', 'German Shepherd', 'Old English Sheepdog', 'Rottweiler', 'Siberian Husky']
        df = pd.DataFrame.from_records(db.read(
            {'$and': [{'animal_type' : 'Dog'}, {'breed': {'$in': mountainRescueList}}, {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gt': 26}}, {'age_upon_outcome_in_weeks': {'$lt': 156}}]}))
    elif(buttonClicked == 'buttonThree'):
        disasterRescueList = ['Bloodhound', 'Doberman Pinsch', 'German Shepherd', 'Golden Retriever', 'Rottweiler']
        df = pd.DataFrame.from_records(db.read(
            {'$and': [{'animal_type' : 'Dog'}, {'breed': {'$in': disasterRescueList}}, {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gt': 20}}, {'age_upon_outcome_in_weeks': {'$lt': 300}}]}))
    else:
        df = pd.DataFrame.from_records(db.read({}))
        df.drop(columns=['_id'],inplace=True)
        
    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    if (viewData is not None):
        return [
            dcc.Graph(
                style = {'width': '1000px', 'height': '500px'},
                figure = px.histogram(dff, x = 'breed')
            )    
        ]
    else:
        return
    
#This callback will highlight a row on the data table when the user selects it via row button
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'derived_viewport_selected_rows')]
)
def update_styles(selected_rows):
    style = []
    if (selected_rows is not None):
        style.append(
            { 'if': {'row_index': selected_rows},
              'background_color': '#15A7DC',
              'border': '1px solid blue'
            }
   )
    return style

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    #Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    #Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10,
               children=[
                    dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
                    dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                        children=[
                            dl.Tooltip(dff.iloc[row,4]),
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(dff.iloc[row,9])
                            ])
                        ])
                ])
    ]

app.run_server(debug = True)


Dash app running on http://127.0.0.1:25703/
